In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
#default_exp search

In [6]:
#export
from food.tools import *
from food.paths import *
from food.psql import *
import requests
from food.qdrant import *

In [7]:
#export
## temp while api is not aperational
from food.clipmodel import *
from PIL import Image

device success 0


In [8]:
#export
foods = read_sql('foods')
# foods = foods.drop(columns = ['clip'])
foods = foods.set_index('id')

In [9]:
#export
def read_image_from_url(url=None):
    response = requests.get(url, stream=True)
    return Image.open(response.raw)


In [10]:
#export
def search_image(url=None,head = 1):
    image_clip = image2clip(read_image_from_url(url)).tolist()
    # image_clip = requests.post(f'http://127.0.0.1:8181/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    df = foods.loc[[r.id for r in results]]
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

    return df.reset_index()

In [11]:
url = 'https://cleanfoodcrush.com/wp-content/uploads/2020/05/cleanfoodcrush-examples-of-conmplex-carbs.png'

In [13]:
url = 'https://api.telegram.org/file/bot5203882708:AAG3G_Y2oZSr-rMG2zoffDVtj3d0KkOFSts/photos/file_1106.jpg'

In [14]:
search_image(url)

,id,description,category,energy,protein,carb,fat,score
0,1104278,"Tea, hot, chai, with milk",Tea,50.0,1.58,8.97,0.98,0.237743


In [10]:
search_image(url,head=200)

,id,description,category,energy,protein,carb,fat,score
0,1104278,"Tea, hot, chai, with milk",Tea,50.0,1.58,8.97,0.98,0.237743
1,1097891,"Coffee creamer, liquid",Cream and cream substitutes,136.0,1.00,11.38,9.97,0.236768
2,1104483,Glug,Liquor and cocktails,96.0,0.10,6.24,0.14,0.233204
3,1097664,"Hot chocolate / Cocoa, made with dry mix and w...","Flavored milk, nonfat",54.0,0.91,11.40,0.55,0.225602
4,1097666,"Hot chocolate / Cocoa, made with dry mix and r...","Flavored milk, reduced fat",96.0,3.79,15.37,2.18,0.224489
...,...,...,...,...,...,...,...,...
195,1104564,"Sports drink, low calorie",Diet sport and energy drinks,8.0,0.05,1.94,0.01,0.193890
196,1100177,"Gravy, beef, fat free","Dips, gravies, other sauces",53.0,3.75,4.81,2.36,0.193839
197,1097533,"Milk, dry, reconstituted, whole","Milk, whole",76.0,4.01,5.86,4.08,0.193783
198,1104212,"Coffee, instant, 50% less caffeine, reconstitu...",Coffee,3.0,0.12,0.61,0.00,0.193779


In [1]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_multiple_foods.ipynb.
Converted 01_search.ipynb.
Converted 0_template copy 2.ipynb.
Converted 0_template copy 3.ipynb.
Converted 0_template copy 4.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted Untitled-1.ipynb.
Converted inference.ipynb.
Converted stats.ipynb.
Converted usda_parsing.ipynb.
